# Crossbar notebook example

Install LTspice and Wine with scripts from the `ltspice_docker` directory:

First setup of docker
- `./docker_firstrun.sh`
- `./docker_service.sh`

To build Docker container
- `./build.sh`
- Follow install instructions when GUI pops up

To start and stop
- `./start.sh`
- `./stop.sh`

To launch LTspice
- `./ltspice.sh <args>`

To remove build and delete Docker container
- `./unbuild.sh`


## Prepare environment

### <mark>Install libraries</mark>


In [ ]:
!pip3 install scipy numpy ltspice ipympl

### <mark>Setup paths</mark>

In [ ]:
# Add folder containing scripts to the path
import sys
sys.path.append('../') ## Change accordingly

# Command to execute LTSpice launch
LTSPICE_INSTANCE = "../../ltspice.sh" ## Change accordingly

In [ ]:
# Store relative path from ltspice docker to current directory
import os
from pathlib import Path

ltspice_absolute = os.path.realpath("../../ltspice.sh")
ltspice_dir = os.path.dirname(ltspice_absolute)
notebook_dir = os.getcwd()

CURRENT_FOLDER = Path(notebook_dir).relative_to(ltspice_dir)
print("CURRENT_FOLDER:", CURRENT_FOLDER)

In [ ]:
# Reload libraries automatically if is being changed on the fly
%load_ext autoreload
%autoreload 2

## Generator

### <mark>Library setup</mark>


In [ ]:
# Library package
import generator as gen

### <mark>Generate memristor cell subcircuit</mark>
Generate cell with specified number of states
- `resistance_states` should be ordered from high resistance to low resistance, where the value of the state is in $\Omega$
- it takes in either integers, or strings with engineering suffix

In [ ]:
my_states = ['1k', '200'] # strings with engineering suffix
# my_states = [1000, 200] # integers are also accepted
gen.generate_cell(resistance_states=my_states, cell_subcircuit="cell_subcircuit")

The following files are generated
- Schematic: `cell_subcircuit.asc`
- Subcircuit: `cell_subcircuit.asy`

In [ ]:
!$LTSPICE_INSTANCE $CURRENT_FOLDER/cell_subcircuit.asc
# !$LTSPICE_INSTANCE $CURRENT_FOLDER/cell_subcircuit.asy

### <mark>Generate crossbar array subcircuit</mark>


Generate a crossbar of specified number of bitlines and wordlines

In [ ]:
gen.generate_cba(wordline=8, bitline=8, filename="crossbar_subcircuit", cell_subcircuit="cell_subcircuit", wrapper="crossbar_wrapper_template")

The following files are generated

- crossbar_subcircuit.asc
- crossbar_subcircuit.asy
- crossbar_wrapper_template.asc

In [ ]:
!$LTSPICE_INSTANCE $CURRENT_FOLDER/crossbar_subcircuit.asc
# !$LTSPICE_INSTANCE $CURRENT_FOLDER/crossbar_subcircuit.asy
# !$LTSPICE_INSTANCE $CURRENT_FOLDER/crossbar_wrapper_template.asc

## Simulator

### <mark>Library setup</mark>

In [ ]:
# Library package
import simulator as sim
sim.set_ltspice(exe=LTSPICE_INSTANCE, dir=CURRENT_FOLDER)

### <mark>Convert LTSpice schematic to generic spice netlist</mark>

In [ ]:
# Convert to a generic netlist
sim.convert_ltspice_sch_to_netlist(filename="crossbar_wrapper_template")

In [ ]:
# Preview netlist with placeholder parameters 
!$LTSPICE_INSTANCE $CURRENT_FOLDER/crossbar_wrapper_template.net

##### Example for 3x3 crossbar array

In [ ]:
# io = {
#     'WORD': [1, 1, 1],
#     'BIT': [0, 0, 0],
#     'G': [
#         [0, 0, 1],
#         [0, 0, 1],
#         [0, 0, 1], 
#     ],
# }
# result = sim.simulate_netlist(wordline=3, bitline=3, io=io, netlist="crossbar_wrapper_template", tmp_id=8)

##### Example for 8x8 crossbar array

In [ ]:
io = {
    'WORD': [1,1,1,1,1,1,1,1],
    'BIT':  [0,0,0,0,0,0,0,0],
    'G': [
        [1,0,0,0,0,0,0,0],
        [0,1,0,0,0,0,0,0],
        [0,0,1,0,0,0,0,0], 
        [0,0,0,1,0,0,0,0], 
        [0,0,0,0,1,0,0,0], 
        [0,0,0,0,0,1,0,0], 
        [0,0,0,0,0,0,1,0], 
        [0,0,0,0,0,0,0,1], 
    ],
}
result = sim.simulate_netlist(wordline=8, bitline=8, io=io, netlist="crossbar_wrapper_template", tmp_id=7)

### <mark>Plot the results</mark>

In [ ]:
print('word_current', result['word_current'])
print('bit_current', result['bit_current'])

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt

plt.figure(figsize=(8,4))

y = result['bit_current']
x = list(map(lambda a: f"bit{a}", range(len(y))))
plt.barh(x, y, label="Bitline current")

y = result['word_current']
x = list(map(lambda a: f"word{a}", range(len(y))))
plt.barh(x, y, label="Wordline current")

plt.title('Figure')
plt.xlabel('Current (A)')
plt.ylabel('Line')
plt.ion()
plt.show()

### Verify Calculation

In [ ]:
import numpy as np
np.matmul(io['WORD'], io['G'])

### The End

Clean up temporary files

In [ ]:
!rm *_tmp*
!rm *.log
!rm *.net*